In [1]:

import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np
import matplotlib
from plotly.subplots import make_subplots
import plotly.graph_objects as go


import warnings
warnings.filterwarnings("ignore")


import gc
import os

import warnings
import pickle

#import pyarrow as pa

from xgboost import XGBClassifier

# from lightgbm import LGBMClassifier
# from lightgbm import early_stopping
# from lightgbm import log_evaluation

from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score, f1_score


import matplotlib.pyplot as plt
# from colorama import Fore, Back, Style


In [2]:

# Set max display options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [3]:
dtypes={'session_id':'category', 
'elapsed_time':np.int32,
    'index':np.int32,
    'event_name':'category',
    'name':'category',
    'level':np.uint8,
    'page':'category',
    'room_coor_x':np.float32,
    'room_coor_y':np.float32,
    'screen_coor_x':np.float32,
    'screen_coor_y':np.float32,
    'hover_duration':np.float32,
     'text':'category',
     'fqid':'category',
     'room_fqid':'category',
     'text_fqid':'category',
     'fullscreen':'category',
     'hq':'category',
     'music':'category',
     'level_group':'category'}
work_path = 'data/predict-student-performance-from-game-play/'
train_df=pd.read_csv(work_path+'train.csv', dtype=dtypes)
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26296946 entries, 0 to 26296945
Data columns (total 20 columns):
 #   Column          Dtype   
---  ------          -----   
 0   session_id      category
 1   index           int32   
 2   elapsed_time    int32   
 3   event_name      category
 4   name            category
 5   level           uint8   
 6   page            category
 7   room_coor_x     float32 
 8   room_coor_y     float32 
 9   screen_coor_x   float32 
 10  screen_coor_y   float32 
 11  hover_duration  float32 
 12  text            category
 13  fqid            category
 14  room_fqid       category
 15  text_fqid       category
 16  fullscreen      category
 17  hq              category
 18  music           category
 19  level_group     category
dtypes: category(12), float32(5), int32(2), uint8(1)
memory usage: 1.1 GB


In [4]:
train_df.head()

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group
0,20090312431273200,0,0,cutscene_click,basic,0,NaN,-413.991394,-159.314682,380.0,494.0,NaN,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,0,0,1,0-4
1,20090312431273200,1,1323,person_click,basic,0,NaN,-413.991394,-159.314682,380.0,494.0,NaN,"Whatcha doing over there, Jo?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4
2,20090312431273200,2,831,person_click,basic,0,NaN,-413.991394,-159.314682,380.0,494.0,NaN,Just talking to Teddy.,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4
3,20090312431273200,3,1147,person_click,basic,0,NaN,-413.991394,-159.314682,380.0,494.0,NaN,I gotta run to my meeting!,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4
4,20090312431273200,4,1863,person_click,basic,0,NaN,-412.991394,-159.314682,381.0,494.0,NaN,"Can I come, Gramps?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4


### Feature engineering

In [19]:
df1 = train_df.query('level_group == "0-4"')
df2 = train_df.query('level_group == "5-12"')
df3 = train_df.query('level_group == "13-22"')

In [20]:
def additional_features(df):
    df = df[['session_id', 'elapsed_time', 'event_name', 'name', 'level',
        'room_coor_x', 'room_coor_y', 'screen_coor_x', 'screen_coor_y',
       'hover_duration', 'text', 'fqid', 'room_fqid', 'text_fqid',
        'level_group']]
    df['time_diff'] = df['elapsed_time'] - df['elapsed_time'].shift(1)
    df['room_coor_x_diff'] = df['room_coor_x'] - df['room_coor_x'].shift(1)
    df['room_coor_y_diff'] = df['room_coor_y'] - df['room_coor_y'].shift(1)
    df['screen_coor_x_diff'] = df['screen_coor_x'] - df['screen_coor_x'].shift(1)
    df['screen_coor_y_diff'] = df['screen_coor_y'] - df['screen_coor_y'].shift(1)

    # text Not nan
    df['text_value'] = df['text'].isna().astype('int')
    return df


In [21]:
df1 = additional_features(df1)
df2 = additional_features(df2)
df3 = additional_features(df3)

In [22]:
# # Load your data
# df = df

# # Empty lists to hold column names
# continuous_vars = []
# discrete_vars = []
# categorical_vars = []

# # Define a threshold for the maximum number of unique values a variable can have to be considered categorical/discrete
# threshold = 10

# # Iterate over columns in DataFrame and determine their variable type
# for column in df.columns:
#     if pd.api.types.is_categorical_dtype(df[column]):
#         categorical_vars.append(column)
#     elif np.issubdtype(df[column].dtype, np.integer):
#         if df[column].nunique() <= threshold:
#             discrete_vars.append(column)
#         else:
#             continuous_vars.append(column)
#     elif np.issubdtype(df[column].dtype, np.floating):
#         continuous_vars.append(column)
#     else:
#         print(f'Variable type of {column} is not clearly identifiable')

# print(f'Continuous variables: {continuous_vars}')
# print(f'Discrete variables: {discrete_vars}')
# print(f'Categorical variables: {categorical_vars}')

In [26]:
CATEGORICAL = [ 'event_name', 'name', 'text', 'fqid', 'room_fqid', 'text_fqid', 'text_value']
NUMERICAL = ['elapsed_time', 'room_coor_x', 'room_coor_y', 'screen_coor_x', 'screen_coor_y', 'hover_duration', 'time_diff', 'room_coor_x_diff', 'room_coor_y_diff', 'screen_coor_x_diff', 'screen_coor_y_diff']

In [27]:
# def feature_engineer(dataset_df):
#     dfs = []
#     for c in CATEGORICAL:
#         tmp = dataset_df.groupby(['session_id','level'])[c].agg('nunique')
#         tmp.name = tmp.name + '_nunique'
#         dfs.append(tmp)
#     for c in NUMERICAL:
#         tmp = dataset_df.groupby(['session_id','level'])[c].agg('mean')
#         tmp.name = tmp.name + '_mean'
#         dfs.append(tmp)
#     for c in NUMERICAL:
#         tmp = dataset_df.groupby(['session_id','level'])[c].agg('std')
#         tmp.name = tmp.name + '_std'
#         dfs.append(tmp)
#     dataset_df = pd.concat(dfs,axis=1)
#     dataset_df = dataset_df.fillna(-1)
#     dataset_df = dataset_df.reset_index()
#     dataset_df = dataset_df.set_index(['session_id', 'level'])
#     # Unstack level and level_group from index
#     dataset_df = dataset_df.unstack('level')

#     # Concatenate column names
#     dataset_df.columns = ['_'.join(map(str, col)) for col in dataset_df.columns]
#     return dataset_df



def feature_engineer(dataset_df):
    # Define aggregation operations for numerical and categorical columns
    agg_numerical = {num_col: ['mean', 'median', 'std', 'sum', 'min', 'max'] for num_col in NUMERICAL}
    agg_categorical = {cat_col: ['nunique','count'] for cat_col in CATEGORICAL}  # 'lambda x:x.value_counts().index[0] if x.nunique() else None' will compute mode

    agg_dict = {**agg_numerical, **agg_categorical}

    # Perform groupby operation for ['session_id', 'level']
    df_level = dataset_df.groupby(['session_id', 'level']).agg(agg_dict)
    df_level.columns = ['_'.join(col).strip() for col in df_level.columns.values]
    df_level = df_level.fillna(-1)
    df_level = df_level.unstack('level')
    df_level.columns = ['_'.join(map(str, col)) for col in df_level.columns]

    

    # Perform groupby operation for ['session_id', 'level_group']
    df_level_group = dataset_df.groupby(['session_id']).agg(agg_dict)
    df_level_group.columns = ['_'.join(col).strip() for col in df_level_group.columns.values]
    df_level_group = df_level_group.fillna(-1)

    # Concatenate the two resulting dataframes
    df_final = pd.concat([df_level, df_level_group], axis=1)

    return df_final

In [28]:
df1_features = feature_engineer(df1)
df2_features = feature_engineer(df2)
df3_features = feature_engineer(df3)



In [29]:
df1_features.shape,  df2_features.shape, df3_features.shape

((23562, 480), (23562, 720), (23562, 880))

In [30]:
df1_features.head()

,elapsed_time_mean_0,elapsed_time_mean_1,elapsed_time_mean_2,elapsed_time_mean_3,elapsed_time_mean_4,elapsed_time_median_0,elapsed_time_median_1,elapsed_time_median_2,elapsed_time_median_3,elapsed_time_median_4,elapsed_time_std_0,elapsed_time_std_1,elapsed_time_std_2,elapsed_time_std_3,elapsed_time_std_4,elapsed_time_sum_0,elapsed_time_sum_1,elapsed_time_sum_2,elapsed_time_sum_3,elapsed_time_sum_4,elapsed_time_min_0,elapsed_time_min_1,elapsed_time_min_2,elapsed_time_min_3,elapsed_time_min_4,elapsed_time_max_0,elapsed_time_max_1,elapsed_time_max_2,elapsed_time_max_3,elapsed_time_max_4,room_coor_x_mean_0,room_coor_x_mean_1,room_coor_x_mean_2,room_coor_x_mean_3,room_coor_x_mean_4,room_coor_x_median_0,room_coor_x_median_1,room_coor_x_median_2,room_coor_x_median_3,room_coor_x_median_4,room_coor_x_std_0,room_coor_x_std_1,room_coor_x_std_2,room_coor_x_std_3,room_coor_x_std_4,room_coor_x_sum_0,room_coor_x_sum_1,room_coor_x_sum_2,room_coor_x_sum_3,room_coor_x_sum_4,room_coor_x_min_0,room_coor_x_min_1,room_coor_x_min_2,room_coor_x_min_3,room_coor_x_min_4,room_coor_x_max_0,room_coor_x_max_1,room_coor_x_max_2,room_coor_x_max_3,room_coor_x_max_4,room_coor_y_mean_0,room_coor_y_mean_1,room_coor_y_mean_2,room_coor_y_mean_3,room_coor_y_mean_4,room_coor_y_median_0,room_coor_y_median_1,room_coor_y_median_2,room_coor_y_median_3,room_coor_y_median_4,room_coor_y_std_0,room_coor_y_std_1,room_coor_y_std_2,room_coor_y_std_3,room_coor_y_std_4,room_coor_y_sum_0,room_coor_y_sum_1,room_coor_y_sum_2,room_coor_y_sum_3,room_coor_y_sum_4,room_coor_y_min_0,room_coor_y_min_1,room_coor_y_min_2,room_coor_y_min_3,room_coor_y_min_4,room_coor_y_max_0,room_coor_y_max_1,room_coor_y_max_2,room_coor_y_max_3,room_coor_y_max_4,screen_coor_x_mean_0,screen_coor_x_mean_1,screen_coor_x_mean_2,screen_coor_x_mean_3,screen_coor_x_mean_4,screen_coor_x_median_0,screen_coor_x_median_1,screen_coor_x_median_2,screen_coor_x_median_3,screen_coor_x_median_4,screen_coor_x_std_0,screen_coor_x_std_1,screen_coor_x_std_2,screen_coor_x_std_3,screen_coor_x_std_4,screen_coor_x_sum_0,screen_coor_x_sum_1,screen_coor_x_sum_2,screen_coor_x_sum_3,screen_coor_x_sum_4,screen_coor_x_min_0,screen_coor_x_min_1,screen_coor_x_min_2,screen_coor_x_min_3,screen_coor_x_min_4,screen_coor_x_max_0,screen_coor_x_max_1,screen_coor_x_max_2,screen_coor_x_max_3,screen_coor_x_max_4,screen_coor_y_mean_0,screen_coor_y_mean_1,screen_coor_y_mean_2,screen_coor_y_mean_3,screen_coor_y_mean_4,screen_coor_y_median_0,screen_coor_y_median_1,screen_coor_y_median_2,screen_coor_y_median_3,screen_coor_y_median_4,screen_coor_y_std_0,screen_coor_y_std_1,screen_coor_y_std_2,screen_coor_y_std_3,screen_coor_y_std_4,screen_coor_y_sum_0,screen_coor_y_sum_1,screen_coor_y_sum_2,screen_coor_y_sum_3,screen_coor_y_sum_4,screen_coor_y_min_0,screen_coor_y_min_1,screen_coor_y_min_2,screen_coor_y_min_3,screen_coor_y_min_4,screen_coor_y_max_0,screen_coor_y_max_1,screen_coor_y_max_2,screen_coor_y_max_3,screen_coor_y_max_4,hover_duration_mean_0,hover_duration_mean_1,hover_duration_mean_2,hover_duration_mean_3,hover_duration_mean_4,hover_duration_median_0,hover_duration_median_1,hover_duration_median_2,hover_duration_median_3,hover_duration_median_4,hover_duration_std_0,hover_duration_std_1,hover_duration_std_2,hover_duration_std_3,hover_duration_std_4,hover_duration_sum_0,hover_duration_sum_1,hover_duration_sum_2,hover_duration_sum_3,hover_duration_sum_4,hover_duration_min_0,hover_duration_min_1,hover_duration_min_2,hover_duration_min_3,hover_duration_min_4,hover_duration_max_0,hover_duration_max_1,hover_duration_max_2,hover_duration_max_3,hover_duration_max_4,time_diff_mean_0,time_diff_mean_1,time_diff_mean_2,time_diff_mean_3,time_diff_mean_4,time_diff_median_0,time_diff_median_1,time_diff_median_2,time_diff_median_3,time_diff_median_4,time_diff_std_0,time_diff_std_1,time_diff_std_2,time_diff_std_3,time_diff_std_4,time_diff_sum_0,time_diff_sum_1,time_diff_sum_2,time_diff_sum_3,time_diff_sum_4,time_diff_min_0,time_diff_min_1,time_diff_min_2,time_diff_m

In [31]:
df2_features.head()

,elapsed_time_mean_5,elapsed_time_mean_6,elapsed_time_mean_7,elapsed_time_mean_8,elapsed_time_mean_9,elapsed_time_mean_10,elapsed_time_mean_11,elapsed_time_mean_12,elapsed_time_median_5,elapsed_time_median_6,elapsed_time_median_7,elapsed_time_median_8,elapsed_time_median_9,elapsed_time_median_10,elapsed_time_median_11,elapsed_time_median_12,elapsed_time_std_5,elapsed_time_std_6,elapsed_time_std_7,elapsed_time_std_8,elapsed_time_std_9,elapsed_time_std_10,elapsed_time_std_11,elapsed_time_std_12,elapsed_time_sum_5,elapsed_time_sum_6,elapsed_time_sum_7,elapsed_time_sum_8,elapsed_time_sum_9,elapsed_time_sum_10,elapsed_time_sum_11,elapsed_time_sum_12,elapsed_time_min_5,elapsed_time_min_6,elapsed_time_min_7,elapsed_time_min_8,elapsed_time_min_9,elapsed_time_min_10,elapsed_time_min_11,elapsed_time_min_12,elapsed_time_max_5,elapsed_time_max_6,elapsed_time_max_7,elapsed_time_max_8,elapsed_time_max_9,elapsed_time_max_10,elapsed_time_max_11,elapsed_time_max_12,room_coor_x_mean_5,room_coor_x_mean_6,room_coor_x_mean_7,room_coor_x_mean_8,room_coor_x_mean_9,room_coor_x_mean_10,room_coor_x_mean_11,room_coor_x_mean_12,room_coor_x_median_5,room_coor_x_median_6,room_coor_x_median_7,room_coor_x_median_8,room_coor_x_median_9,room_coor_x_median_10,room_coor_x_median_11,room_coor_x_median_12,room_coor_x_std_5,room_coor_x_std_6,room_coor_x_std_7,room_coor_x_std_8,room_coor_x_std_9,room_coor_x_std_10,room_coor_x_std_11,room_coor_x_std_12,room_coor_x_sum_5,room_coor_x_sum_6,room_coor_x_sum_7,room_coor_x_sum_8,room_coor_x_sum_9,room_coor_x_sum_10,room_coor_x_sum_11,room_coor_x_sum_12,room_coor_x_min_5,room_coor_x_min_6,room_coor_x_min_7,room_coor_x_min_8,room_coor_x_min_9,room_coor_x_min_10,room_coor_x_min_11,room_coor_x_min_12,room_coor_x_max_5,room_coor_x_max_6,room_coor_x_max_7,room_coor_x_max_8,room_coor_x_max_9,room_coor_x_max_10,room_coor_x_max_11,room_coor_x_max_12,room_coor_y_mean_5,room_coor_y_mean_6,room_coor_y_mean_7,room_coor_y_mean_8,room_coor_y_mean_9,room_coor_y_mean_10,room_coor_y_mean_11,room_coor_y_mean_12,room_coor_y_median_5,room_coor_y_median_6,room_coor_y_median_7,room_coor_y_median_8,room_coor_y_median_9,room_coor_y_median_10,room_coor_y_median_11,room_coor_y_median_12,room_coor_y_std_5,room_coor_y_std_6,room_coor_y_std_7,room_coor_y_std_8,room_coor_y_std_9,room_coor_y_std_10,room_coor_y_std_11,room_coor_y_std_12,room_coor_y_sum_5,room_coor_y_sum_6,room_coor_y_sum_7,room_coor_y_sum_8,room_coor_y_sum_9,room_coor_y_sum_10,room_coor_y_sum_11,room_coor_y_sum_12,room_coor_y_min_5,room_coor_y_min_6,room_coor_y_min_7,room_coor_y_min_8,room_coor_y_min_9,room_coor_y_min_10,room_coor_y_min_11,room_coor_y_min_12,room_coor_y_max_5,room_coor_y_max_6,room_coor_y_max_7,room_coor_y_max_8,room_coor_y_max_9,room_coor_y_max_10,room_coor_y_max_11,room_coor_y_max_12,screen_coor_x_mean_5,screen_coor_x_mean_6,screen_coor_x_mean_7,screen_coor_x_mean_8,screen_coor_x_mean_9,screen_coor_x_mean_10,screen_coor_x_mean_11,screen_coor_x_mean_12,screen_coor_x_median_5,screen_coor_x_median_6,screen_coor_x_median_7,screen_coor_x_median_8,screen_coor_x_median_9,screen_coor_x_median_10,screen_coor_x_median_11,screen_coor_x_median_12,screen_coor_x_std_5,screen_coor_x_std_6,screen_coor_x_std_7,screen_coor_x_std_8,screen_coor_x_std_9,screen_coor_x_std_10,screen_coor_x_std_11,screen_coor_x_std_12,screen_coor_x_sum_5,screen_coor_x_sum_6,screen_coor_x_sum_7,screen_coor_x_sum_8,screen_coor_x_sum_9,screen_coor_x_sum_10,screen_coor_x_sum_11,screen_coor_x_sum_12,screen_coor_x_min_5,screen_coor_x_min_6,screen_coor_x_min_7,screen_coor_x_min_8,screen_coor_x_min_9,screen_coor_x_min_10,screen_coor_x_min_11,screen_coor_x_min_12,screen_coor_x_max_5,screen_coor_x_max_6,screen_coor_x_max_7,screen_coor_x_max_8,screen_coor_x_max_9,screen_coor_x_max_10,screen_coor_x_max_11,screen_coor_x_max_12,screen_coor_y_mean_5,screen_coor_y_mean_6,screen_coor_y_mean_7,screen_coor_y_mean_8,screen_coor_y_mean_9,screen_coor_y_mean_10,screen_coor_y_mean_11,screen_coor_y_mean_12,screen_coor_y_median_5,scree

In [32]:
df3_features.head()

,elapsed_time_mean_13,elapsed_time_mean_14,elapsed_time_mean_15,elapsed_time_mean_16,elapsed_time_mean_17,elapsed_time_mean_18,elapsed_time_mean_19,elapsed_time_mean_20,elapsed_time_mean_21,elapsed_time_mean_22,elapsed_time_median_13,elapsed_time_median_14,elapsed_time_median_15,elapsed_time_median_16,elapsed_time_median_17,elapsed_time_median_18,elapsed_time_median_19,elapsed_time_median_20,elapsed_time_median_21,elapsed_time_median_22,elapsed_time_std_13,elapsed_time_std_14,elapsed_time_std_15,elapsed_time_std_16,elapsed_time_std_17,elapsed_time_std_18,elapsed_time_std_19,elapsed_time_std_20,elapsed_time_std_21,elapsed_time_std_22,elapsed_time_sum_13,elapsed_time_sum_14,elapsed_time_sum_15,elapsed_time_sum_16,elapsed_time_sum_17,elapsed_time_sum_18,elapsed_time_sum_19,elapsed_time_sum_20,elapsed_time_sum_21,elapsed_time_sum_22,elapsed_time_min_13,elapsed_time_min_14,elapsed_time_min_15,elapsed_time_min_16,elapsed_time_min_17,elapsed_time_min_18,elapsed_time_min_19,elapsed_time_min_20,elapsed_time_min_21,elapsed_time_min_22,elapsed_time_max_13,elapsed_time_max_14,elapsed_time_max_15,elapsed_time_max_16,elapsed_time_max_17,elapsed_time_max_18,elapsed_time_max_19,elapsed_time_max_20,elapsed_time_max_21,elapsed_time_max_22,room_coor_x_mean_13,room_coor_x_mean_14,room_coor_x_mean_15,room_coor_x_mean_16,room_coor_x_mean_17,room_coor_x_mean_18,room_coor_x_mean_19,room_coor_x_mean_20,room_coor_x_mean_21,room_coor_x_mean_22,room_coor_x_median_13,room_coor_x_median_14,room_coor_x_median_15,room_coor_x_median_16,room_coor_x_median_17,room_coor_x_median_18,room_coor_x_median_19,room_coor_x_median_20,room_coor_x_median_21,room_coor_x_median_22,room_coor_x_std_13,room_coor_x_std_14,room_coor_x_std_15,room_coor_x_std_16,room_coor_x_std_17,room_coor_x_std_18,room_coor_x_std_19,room_coor_x_std_20,room_coor_x_std_21,room_coor_x_std_22,room_coor_x_sum_13,room_coor_x_sum_14,room_coor_x_sum_15,room_coor_x_sum_16,room_coor_x_sum_17,room_coor_x_sum_18,room_coor_x_sum_19,room_coor_x_sum_20,room_coor_x_sum_21,room_coor_x_sum_22,room_coor_x_min_13,room_coor_x_min_14,room_coor_x_min_15,room_coor_x_min_16,room_coor_x_min_17,room_coor_x_min_18,room_coor_x_min_19,room_coor_x_min_20,room_coor_x_min_21,room_coor_x_min_22,room_coor_x_max_13,room_coor_x_max_14,room_coor_x_max_15,room_coor_x_max_16,room_coor_x_max_17,room_coor_x_max_18,room_coor_x_max_19,room_coor_x_max_20,room_coor_x_max_21,room_coor_x_max_22,room_coor_y_mean_13,room_coor_y_mean_14,room_coor_y_mean_15,room_coor_y_mean_16,room_coor_y_mean_17,room_coor_y_mean_18,room_coor_y_mean_19,room_coor_y_mean_20,room_coor_y_mean_21,room_coor_y_mean_22,room_coor_y_median_13,room_coor_y_median_14,room_coor_y_median_15,room_coor_y_median_16,room_coor_y_median_17,room_coor_y_median_18,room_coor_y_median_19,room_coor_y_median_20,room_coor_y_median_21,room_coor_y_median_22,room_coor_y_std_13,room_coor_y_std_14,room_coor_y_std_15,room_coor_y_std_16,room_coor_y_std_17,room_coor_y_std_18,room_coor_y_std_19,room_coor_y_std_20,room_coor_y_std_21,room_coor_y_std_22,room_coor_y_sum_13,room_coor_y_sum_14,room_coor_y_sum_15,room_coor_y_sum_16,room_coor_y_sum_17,room_coor_y_sum_18,room_coor_y_sum_19,room_coor_y_sum_20,room_coor_y_sum_21,room_coor_y_sum_22,room_coor_y_min_13,room_coor_y_min_14,room_coor_y_min_15,room_coor_y_min_16,room_coor_y_min_17,room_coor_y_min_18,room_coor_y_min_19,room_coor_y_min_20,room_coor_y_min_21,room_coor_y_min_22,room_coor_y_max_13,room_coor_y_max_14,room_coor_y_max_15,room_coor_y_max_16,room_coor_y_max_17,room_coor_y_max_18,room_coor_y_max_19,room_coor_y_max_20,room_coor_y_max_21,room_coor_y_max_22,screen_coor_x_mean_13,screen_coor_x_mean_14,screen_coor_x_mean_15,screen_coor_x_mean_16,screen_coor_x_mean_17,screen_coor_x_mean_18,screen_coor_x_mean_19,screen_coor_x_mean_20,screen_coor_x_mean_21,screen_coor_x_mean_22,screen_coor_x_median_13,screen_coor_x_median_14,screen_coor_x_median_15,screen_coor_x_median_16,screen_coor_x_median_17,screen_coor_x_median_18,screen_coor_x_median_19,screen_coor

### Possible additional features

In [ ]:
df.head(5)#.groupby(['session_id','text_fqid'])['time_diff'].count()


,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group,time_diff,room_coor_x_diff,room_coor_y_diff,screen_coor_x_diff,screen_coor_y_diff
0,20090312431273200,0,0,cutscene_click,basic,0,NaN,-413.991394,-159.314682,380.0,494.0,NaN,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,0,0,1,0-4,0.0,NaN,NaN,NaN,NaN
1,20090312431273200,1,1323,person_click,basic,0,NaN,-413.991394,-159.314682,380.0,494.0,NaN,"Whatcha doing over there, Jo?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4,1323.0,0.0,0.0,0.0,0.0
2,20090312431273200,2,831,person_click,basic,0,NaN,-413.991394,-159.314682,380.0,494.0,NaN,Just talking to Teddy.,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4,-492.0,0.0,0.0,0.0,0.0
3,20090312431273200,3,1147,person_click,basic,0,NaN,-413.991394,-159.314682,380.0,494.0,NaN,I gotta run to my meeting!,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4,316.0,0.0,0.0,0.0,0.0
4,20090312431273200,4,1863,person_click,basic,0,NaN,-412.991394,-159.314682,381.0,494.0,NaN,"Can I come, Gramps?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4,716.0,1.0,0.0,1.0,0.0


In [ ]:
df.groupby(['session_id','room_fqid'])['time_diff'].sum()

session_id         room_fqid                   
20090312431273200  tunic.capitol_0.hall            32422.0
                   tunic.capitol_1.hall                0.0
                   tunic.capitol_2.hall                0.0
                   tunic.drycleaner.frontdesk          0.0
                   tunic.flaghouse.entry               0.0
                                                    ...   
22100221145014656  tunic.humanecology.frontdesk        0.0
                   tunic.kohlcenter.halloffame     57931.0
                   tunic.library.frontdesk             0.0
                   tunic.library.microfiche            0.0
                   tunic.wildlife.center               0.0
Name: time_diff, Length: 447678, dtype: float64

In [ ]:

# time between text values 
df.groupby(['session_id','text_value'])['time_diff'].mean()

session_id         text_value
20090312431273200  0             1082.852459
                   1             1250.543689
20090312433251036  0            -1525.262295
                   1             1691.448718
20090312455206810  0            -2356.868421
                                    ...     
22100217104993650  1             2159.809160
22100219442786200  0            -5732.377049
                   1             1242.858491
22100221145014656  0              -67.319444
                   1             1785.431655
Name: time_diff, Length: 47124, dtype: float64

In [ ]:
# averange coordinate distance between text values 
df.groupby(['session_id','text_value'])['room_coor_x'].mean()

session_id         text_value
20090312431273200  0              82.651031
                   1             -41.733669
20090312433251036  0            -128.311600
                   1             -44.912571
20090312455206810  0            -184.254013
                                    ...    
22100217104993650  1              11.843848
22100219442786200  0             -28.325773
                   1             -70.563507
22100221145014656  0               8.662521
                   1              -7.010103
Name: room_coor_x, Length: 47124, dtype: float32

### Train labels 

In [16]:
train_label = pd.read_csv(work_path+'train_labels.csv')
train_label['session'] = train_label.session_id.apply(lambda x: int(x.split('_')[0]) )
train_label['q'] = train_label.session_id.apply(lambda x: int(x.split('_')[-1][1:]) )
print( 'shape of label dataset is:',train_label.shape )


shape of label dataset is: (424116, 4)


### XGBoost model

In [33]:
#estimators_xgb = [498, 448, 378, 364, 405, 495, 456, 249, 384, 405, 356, 262, 484, 381, 392, 248 ,248, 345]

xgb_params = {
        'booster': 'gbtree',
        'tree_method': 'hist',
        'objective': 'binary:logistic',
        'eval_metric':'logloss',
        'learning_rate': 0.02,
        'alpha': 8,
        'max_depth': 4,
        'subsample':0.8,
        'colsample_bytree': 0.5,
        'seed': 42
        }

In [35]:
warnings.filterwarnings("ignore")
targets = pd.read_csv(work_path+'train_labels.csv')
targets['session'] = targets.session_id.apply(lambda x: int(x.split('_')[0]))
targets['q'] = targets.session_id.apply(lambda x: int(x.split('_')[-1][1:]))
pred_xgb = np.zeros((df1_features.shape[0],18))     


In [42]:
warnings.filterwarnings("ignore")
targets = pd.read_csv(work_path+'train_labels.csv')
targets['session'] = targets.session_id.apply(lambda x: int(x.split('_')[0]))
targets['q'] = targets.session_id.apply(lambda x: int(x.split('_')[-1][1:]))
pred_xgb = np.zeros((df1_features.shape[0],18))     

for t in range(1,19):
    # USE THIS TRAIN DATA WITH THESE QUESTIONS
    if t<=3: 
        grp = '0-4'
        df = df1_features
        FEATURES = df1_features.columns

    elif t<=13: 
        grp = '5-12'
        df = df2_features
        FEATURES = df2_features.columns

    elif t<=22: 
        grp = '13-22'
        df = df3_features
        FEATURES = df3_features.columns
        
    xgb_params['n_estimators'] = 300 #estimators_xgb[t-1]
     
    # TRAIN DATA
    train_users = df.index.values.astype('int')
    train_y = targets.loc[targets.q==t].set_index('session').loc[train_users] 

    clf =  XGBClassifier(**xgb_params)
    clf.fit(df[FEATURES].astype('float32'), train_y['correct'], verbose = 0)
    clf.save_model(f'XG_boost_models/XGB_question{t}.xgb')

    
    print(f'model XGB saved for question {t}')
    #print(f'model XGB saved for question {t} with iterations = {estimators_xgb[t-1]}')

model XGB saved for question 1
model XGB saved for question 2
model XGB saved for question 3
model XGB saved for question 4
model XGB saved for question 5
model XGB saved for question 6
model XGB saved for question 7
model XGB saved for question 8
model XGB saved for question 9
model XGB saved for question 10
model XGB saved for question 11
model XGB saved for question 12
model XGB saved for question 13
model XGB saved for question 14
model XGB saved for question 15
model XGB saved for question 16
model XGB saved for question 17
model XGB saved for question 18
